<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2011/Deep_Learning_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd              # Untuk memproses data dalam bentuk DataFrame
import numpy as np               # Untuk operasi numerik
import torch                     # Library utama untuk deep learning
from torch import nn             # Module untuk membangun neural network
from torch.utils.data import Dataset, DataLoader  # Untuk dataset dan data batching
from sklearn.model_selection import train_test_split  # Untuk membagi data menjadi training dan testing
from sklearn.preprocessing import StandardScaler      # Untuk normalisasi fitur
from sklearn.metrics import accuracy_score           # Untuk menghitung akurasi


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load dataset
file_path = '/content/drive/MyDrive/Week 11/heart.csv'  # Path ke dataset
data = pd.read_csv(file_path)      # Membaca dataset ke dalam DataFrame

# Menampilkan overview dari dataset
print("Dataset Overview:\n", data.head())  # Menampilkan 5 baris pertama dari dataset
print("\nDataset Info:\n")
data.info()  # Menampilkan informasi dataset seperti kolom dan tipe data


Dataset Overview:
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  

Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   

In [4]:
# Memisahkan fitur (X) dan target (y)
X = data.drop(columns=['target'])  # Menghapus kolom target dari dataset
y = data['target']                 # Menyimpan kolom target ke variabel y

# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()          # Inisialisasi scaler
X_scaled = scaler.fit_transform(X) # Menormalisasi semua fitur

# Membagi dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mengonversi data ke dalam bentuk tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Tensor fitur training
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)    # Tensor fitur testing
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)  # Tensor target training
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)    # Tensor target testing


In [5]:
# Mendefinisikan dataset custom
class HeartDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features  # Menyimpan fitur
        self.labels = labels      # Menyimpan target

    def __len__(self):
        return len(self.features)  # Mengembalikan panjang dataset

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]  # Mengembalikan item berdasarkan indeks

# Membuat objek dataset untuk training dan testing
dataset_train = HeartDataset(X_train_tensor, y_train_tensor)
dataset_test = HeartDataset(X_test_tensor, y_test_tensor)


In [6]:
# Mendefinisikan MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPClassifier, self).__init__()
        layers = []
        in_features = input_size

        # Membuat hidden layers
        for hidden_units in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_units))  # Linear Layer
            layers.append(activation_fn)  # Activation Function
            in_features = hidden_units

        # Output layer
        layers.append(nn.Linear(in_features, 2))  # Binary classification (2 classes)
        self.model = nn.Sequential(*layers)  # Menggabungkan semua layer ke dalam Sequential

    def forward(self, x):
        return self.model(x)  # Mendefinisikan forward pass


In [7]:
# Fungsi untuk melatih model
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.train()  # Set model ke mode training
    for epoch in range(num_epochs):  # Iterasi setiap epoch
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()  # Reset gradient sebelumnya
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Hitung loss
            loss.backward()  # Backward pass (menghitung gradien)
            optimizer.step()  # Update parameter
            running_loss += loss.item() * inputs.size(0)  # Total loss
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Fungsi untuk evaluasi model
def evaluate_model(model, dataloader):
    model.eval()  # Set model ke mode evaluasi
    predictions, targets = [], []
    with torch.no_grad():  # Tidak menghitung gradien
        for inputs, labels in dataloader:
            outputs = model(inputs)  # Forward pass
            _, preds = torch.max(outputs, 1)  # Mengambil prediksi kelas
            predictions.extend(preds.numpy())
            targets.extend(labels.numpy())
    accuracy = accuracy_score(targets, predictions)
    return accuracy


In [8]:
# Inisialisasi eksperimen
input_size = X_train.shape[1]  # Jumlah fitur input
results = []  # Menyimpan hasil eksperimen

# Looping eksperimen
for hidden_layers in [[4], [8, 4], [16, 8, 4]]:
    for activation_fn in [nn.ReLU(), nn.Sigmoid(), nn.Tanh()]:
        for lr in [0.1, 0.01, 0.001]:
            for epochs in [10, 50, 100]:
                for batch_size in [32, 64, 128]:
                    # Membuat model
                    model = MLPClassifier(input_size=input_size, hidden_layers=hidden_layers, activation_fn=activation_fn)

                    # Membuat DataLoader
                    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
                    test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

                    # Inisialisasi loss function dan optimizer
                    criterion = nn.CrossEntropyLoss()  # Digunakan untuk klasifikasi
                    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

                    # Melatih model
                    print(f"Training model with hidden_layers={hidden_layers}, activation_fn={activation_fn}, "
                          f"lr={lr}, epochs={epochs}, batch_size={batch_size}")
                    train_model(model, train_loader, criterion, optimizer, epochs)

                    # Evaluasi model
                    acc = evaluate_model(model, test_loader)
                    print(f"Accuracy: {acc:.4f}")

                    # Menyimpan hasil eksperimen
                    results.append({
                        "hidden_layers": hidden_layers,
                        "activation_fn": activation_fn.__class__.__name__,
                        "lr": lr,
                        "epochs": epochs,
                        "batch_size": batch_size,
                        "accuracy": acc
                    })


Streaming output truncated to the last 5000 lines.
Epoch 82/100, Loss: 0.0264
Epoch 83/100, Loss: 0.0262
Epoch 84/100, Loss: 0.0262
Epoch 85/100, Loss: 0.0260
Epoch 86/100, Loss: 0.0257
Epoch 87/100, Loss: 0.0256
Epoch 88/100, Loss: 0.0255
Epoch 89/100, Loss: 0.0253
Epoch 90/100, Loss: 0.0251
Epoch 91/100, Loss: 0.0250
Epoch 92/100, Loss: 0.0249
Epoch 93/100, Loss: 0.0248
Epoch 94/100, Loss: 0.0247
Epoch 95/100, Loss: 0.0246
Epoch 96/100, Loss: 0.0244
Epoch 97/100, Loss: 0.0243
Epoch 98/100, Loss: 0.0242
Epoch 99/100, Loss: 0.0241
Epoch 100/100, Loss: 0.0239
Accuracy: 0.9805
Training model with hidden_layers=[8, 4], activation_fn=Tanh(), lr=0.001, epochs=10, batch_size=32
Epoch 1/10, Loss: 0.6590
Epoch 2/10, Loss: 0.6176
Epoch 3/10, Loss: 0.5681
Epoch 4/10, Loss: 0.5146
Epoch 5/10, Loss: 0.4656
Epoch 6/10, Loss: 0.4247
Epoch 7/10, Loss: 0.3955
Epoch 8/10, Loss: 0.3745
Epoch 9/10, Loss: 0.3607
Epoch 10/10, Loss: 0.3504
Accuracy: 0.7951
Training model with hidden_layers=[8, 4], activatio

In [11]:
# Mengonversi hasil eksperimen ke DataFrame untuk analisis
results_df = pd.DataFrame(results)

# Menampilkan hasil terbaik
best_result = results_df.loc[results_df['accuracy'].idxmax()]
print("\nBest Hyperparameter Combination:\n", best_result)

# Menyimpan hasil eksperimen ke file CSV
results_df.to_csv('/content/drive/MyDrive/Week 11/deep experiment_results.csv', index=False)



Best Hyperparameter Combination:
 hidden_layers    [16, 8, 4]
activation_fn          Tanh
lr                     0.01
epochs                   50
batch_size               32
accuracy                1.0
Name: 228, dtype: object
